# DATA DOWNLOADING AND CLEANING

In [1]:
import psycopg2
from psycopg2.extras import RealDictCursor
import pandas as pd
from multiprocessing import Process
import datetime as dt

In [18]:
conn = psycopg2.connect(host="qc-aurora.coh4objazhte.us-east-1.rds.amazonaws.com", database='qcdb', user='Capstone_User',
password='capstoneFall2019!', port="5432")

In [5]:
cur = conn.cursor(cursor_factory=RealDictCursor)
target = cur.execute('SELECT * FROM text_data.classifier_hits_w_columnname ORDER BY date LIMIT 10')
rows = cur.fetchall()
data = pd.DataFrame(rows)

In [6]:
columns=data.columns
len(columns)

313

In [8]:
columns[data.dtypes==object]   #Removing columns with type other than int

Index(['company_name', 'identifier', 'perm_id'], dtype='object')

In [9]:
final_column=[cols for cols in columns if cols not in ['company_name', 'identifier', 'perm_id','date']]

In [10]:
len(final_column)

309

In [11]:
#Creating SELECT query to aggregate all features day-wise

a=""
for col in final_column:
    a=a+'SUM('+str(col)+') as '+str(col)+', '
    
a=a[:-2]

In [14]:
query = "SELECT date, "+a+" FROM text_data.classifier_hits_w_columnname GROUP BY date"

In [16]:
cur = conn.cursor(cursor_factory=RealDictCursor)
target = cur.execute(query)
rows = cur.fetchall()
data = pd.DataFrame(rows)

In [19]:
#Downloading regime data

cur = conn.cursor(cursor_factory=RealDictCursor)
cur.execute('SELECT * FROM text_data.regime')
rows = cur.fetchall()
regime = pd.DataFrame(rows)

In [21]:
#Convert date column in regime data to date format
regime.date = pd.to_datetime(regime.date)

In [22]:
#Adding year and week number of year to regime to help merge with feature data

wnn = []
for i in range(len(regime)):
    wnn.append(regime.date[i].weekofyear)
regime['week_no'] = wnn

ynn = []
for i in range(len(regime)):
    ynn.append(regime.date[i].year)
regime['year'] = ynn

In [23]:
#Convert date column in feature data to date format

data.date =  pd.to_datetime(data.date)

In [24]:
#Performing manipulations to week of year in feature data to ensure the right features are merged with the right regimes

wn = []
yn = []
for i in range(len(data)):
    if data.date[i].weekday() == 5 or data.date[i].weekday() == 6:
        if data.date[i].month == 12 and data.date[i].weekofyear == 1:
            yn.append(data.date[i].year+1)
            wn.append(data.dat[i].weekofyear)
        elif data.date[i].month == 1 and data.date[i].weekofyear == 52:
            yn.append(data.date[i].year)
            wn.append(1)
        elif data.date[i].month == 12 and data.date[i].weekofyear == 52:
            yn.append(data.date[i].year+1)
            wn.append(1)
        else:
            yn.append(data.date[i].year)
            wn.append(data.date[i].weekofyear+1)
    else:
        if data.date[i].month == 12 and data.date[i].weekofyear == 1:
            yn.append(data.date[i].year+1)
            wn.append(1)
        elif data.date[i].month == 12 and data.date[i].weekofyear == 53 and (data.date[i].year == 2015 or data.date[i].year == 2009):
            yn.append(data.date[i].year+1)
            wn.append(data.date[i].weekofyear)
        else:
            yn.append(data.date[i].year)
            wn.append(data.date[i].weekofyear)        
data['week_no'] = wn
data['year'] = yn

In [25]:
#Merging features and regime

final_data = pd.merge(data,regime,on=['year','week_no'],how='left')

In [26]:
#Regime data is available unitl 2016, but feature data is available until 2018. Deleting data after 2016

final_data = final_data[final_data['date_x']<=pd.to_datetime('20180316')]

In [27]:
len(final_data)

4273

In [29]:
final_data.head()

,accounting_changes,accounting_red_flags,alternative_technology_development,assembly_bottleneck,available_seat_km,bad_delivery_quality,capacity_decrease,capacity_increase,capacity_shortage,capital_reinvestment,...,weak_strategy_bool,work_stoppage,yield_curve,yield_curve_flattening,yield_curve_steepening,week_no,year,date_y,mkt_ret_regimes,three_factor_regimes
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,16,1999,1999-04-23,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,29,1999,1999-07-23,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,4.0,16.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,42,1999,1999-10-22,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,17,2000,2000-04-28,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,30,2000,2000-07-28,1.0,1.0


## Verifying if each feature data merged with the right regime data

In [30]:
week_day = []
for i in range(len(final_data)):
    week_day.append(final_data.date_x[i].weekday())
final_data['week_day.x'] = week_day

### Friday

In [31]:
friday = final_data[final_data['week_day.x'] == 4][['date_x','date_y','year','week_no','mkt_ret_regimes']]

x = []
for i in range(len(friday)):
    if friday.iloc[i,].date_y == friday.iloc[i,].date_x:
        x.append(0)
    else:
        x.append(1)               

In [32]:
sum(x)

0

### Saturday

In [33]:
saturday = final_data[final_data['week_day.x'] == 5][['date_x','date_y','year','week_no','mkt_ret_regimes']]

sat = []
for i in range(len(saturday)):
    if (saturday.iloc[i,].date_y - saturday.iloc[i,].date_x) == pd.to_timedelta('6 days'):
        sat.append(0)
    else:
        sat.append(1) 

In [34]:
sum(sat)

0

### Sunday

In [35]:
sunday = final_data[final_data['week_day.x'] == 6][['date_x','date_y','year','week_no','mkt_ret_regimes']]

sun = []
for i in range(len(sunday)):
    if (sunday.iloc[i,].date_y - sunday.iloc[i,].date_x) == pd.to_timedelta('5 days'):
        sun.append(0)
    else:
        sun.append(1) 

In [36]:
sum(sun)

0

### Monday

In [37]:
monday = final_data[final_data['week_day.x'] == 0][['date_x','date_y','year','week_no','mkt_ret_regimes']]

mon = []
for i in range(len(monday)):
    if (monday.iloc[i,].date_y - monday.iloc[i,].date_x) == pd.to_timedelta('4 days'):
        mon.append(0)
    else:
        mon.append(1) 

In [39]:
sum(mon)

0

### Tuesday

In [40]:
tuesday = final_data[final_data['week_day.x'] == 1][['date_x','date_y','year','week_no','mkt_ret_regimes']]

tue = []
for i in range(len(tuesday)):
    if (tuesday.iloc[i,].date_y - tuesday.iloc[i,].date_x) == pd.to_timedelta('3 days'):
        tue.append(0)
    else:
        tue.append(1) 

In [41]:
sum(tue)

0

### Wednesday

In [42]:
wednesday = final_data[final_data['week_day.x'] == 2][['date_x','date_y','year','week_no','mkt_ret_regimes']]

wed = []
for i in range(len(wednesday)):
    if (wednesday.iloc[i,].date_y - wednesday.iloc[i,].date_x) == pd.to_timedelta('2 days'):
        wed.append(0)
    else:
        wed.append(1) 

In [43]:
sum(wed)

0

### Thursday

In [44]:
thursday = final_data[final_data['week_day.x'] == 3][['date_x','date_y','year','week_no','mkt_ret_regimes']]

thu = []
for i in range(len(thursday)):
    if (thursday.iloc[i,].date_y - thursday.iloc[i,].date_x) == pd.to_timedelta('1 days'):
        thu.append(0)
    else:
        thu.append(1) 

In [45]:
sum(thu)

0

In [46]:
#verifying all data points have been verified

len(monday)+len(tuesday)+len(wednesday)+len(thursday)+len(friday)+len(saturday)+len(sunday)

4273

In [47]:
len(final_data)

4273

In [48]:
#Dropping non-numerical columns to aggrgate data week-wise

final_data = final_data.drop(columns = ['date_x','date_y','week_day.x'])

final_data = final_data.drop(columns = ['three_factor_regimes'])

In [49]:
#Aggregating data week-wise

final_data = final_data.groupby(['year','week_no']).sum()
final_data

accounting_changes  accounting_red_flags  \
year week_no                                             
1999 16                      0.0                   0.0   
     29                      0.0                   0.0   
     42                      0.0                   0.0   
2000 17                      0.0                   0.0   
     30                      0.0                   0.0   
     43                      0.0                   0.0   
2001 4                       0.0                   0.0   
     17                      0.0                   0.0   
     30                      0.0                   0.0   
     39                      0.0                   0.0   
     43                      0.0                   0.0   
2002 4                       0.0                   0.0   
     17                      0.0                   0.0   
     51                      0.0                   0.0   
2003 3                       0.0                  76.0   
     4                      12.0                 160.0   
     5                       0.0                   0.0   
     6                       0.0                  16.0   
     7                      44.0                  72.0   
     8                       0.0                   0.0   
     9                       0.0                  12.0   
     10                      0.0                   0.0   
     11                      0.0                   0.0   
     12                      0.0                  60.0   
     13                      0.0                   0.0   
     14                      0.0                   0.0   
     15                      0.0                   0.0   
     16                      0.0                  24.0   
     17                      0.0                 188.0   
     18                     16.0                  96.0   
...                          ...                   ...   
2017 34                      0.0                  22.0   
     35                      0.0                   2.0   
     36                      2.0                  86.0   
     37                      2.0                  52.0   
     38                      0.0                  14.0   
     39                      0.0                  14.0   
     40                      0.0                  16.0   
     41                      0.0                  16.0   
     42                      0.0                   6.0   
     43                      2.0                  58.0   
     44                      6.0                  82.0   
     45                      0.0                 152.0   
     46                      0.0                  90.0   
     47                      0.0                  30.0   
     48                      0.0                 126.0   
     49                      2.0                  82.0   
     50                      0.0                   4.0   
     51                      0.0                  64.0   
     52                      0.0                   0.0   
2018 1                       0.0                   0.0   
     2                       3.0                  39.0   
     3                       0.0                   6.0   
     4                       0.0                  18.0   
     5                       0.0                  39.0   
     6                       0.0                  45.0   
     7                      15.0                  60.0   
     8                       0.0                  25.0   
     9                       0.0                  42.0   
     10                      0.0                 129.0   
     11                      0.0                  27.0   

              alternative_technology_development  assembly_bottleneck  \
year week_no                                                            
1999 16                                      0.0                  0.0   
     29                                      0.0                  0.0   
     42                                      0.0                  0

In [50]:
final_data = final_data.reset_index()

In [52]:
final_data = final_data.drop(columns = ['year','week_no'])

In [53]:
final_data

,accounting_changes,accounting_red_flags,alternative_technology_development,assembly_bottleneck,available_seat_km,bad_delivery_quality,capacity_decrease,capacity_increase,capacity_shortage,capital_reinvestment,...,transportation_disruption,tsr_ltip_drivers,unstable_demand,weak_demand_bool,weak_strategy_bool,work_stoppage,yield_curve,yield_curve_flattening,yield_curve_steepening,mkt_ret_regimes
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,4.0,16.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,...,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,1.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0,0.0,1.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,1.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,1.0
